# Cluster/local configs

In [ ]:
# If on cluster, cwd is '/rds/general/user/nz423', not the project directory, breaking imports
import os
import sys
from pathlib import Path

project_dir = Path(os.getcwd()) / "exploring-hydra-boosting"
if "rds" not in project_dir.parts:
    project_dir = project_dir.parent
    datasets_dir = project_dir.parent.parent / "Data"
else:
    datasets_dir = project_dir / "DATASETS"
    if str(project_dir) not in sys.path:
        sys.path.append(str(project_dir))
    
class Config:
    project_dir = project_dir
    TSER_dir = datasets_dir / "TSER"
    BigTSC_dir = datasets_dir / "BigTSC"
    UTSC_dir = datasets_dir / "UTSC"
    results_dir = project_dir / "results_TESTING"

print(Config.project_dir)
print(Config.TSER_dir)
print(Config.BigTSC_dir)
print(Config.UTSC_dir)
print(Config.results_dir)

# Simple Wrapper no gridsearch

In [ ]:
# class TSMLWrapperHydraBoost(RegressorMixin, BaseTimeSeriesEstimator):
    
#     def __init__(self, **kwargs):
#         super(TSMLWrapperHydraBoost, self).__init__()
#         self.hydraboost = HydraBoost(
#             n_layers=1,
#             init_n_kernels=8,
#             init_n_groups=64,
#             n_kernels=8,
#             n_groups=64,
#             max_num_channels=3,
#             hydra_batch_size=10000,
#             l2_reg=10,
#             l2_ghat=0.1,
#             boost_lr=1,
#             train_top_at = [0, 5, 10],
#             **kwargs
#         )
        

#     def fit(self, X: np.ndarray, y: np.ndarray) -> object:
#         """Fit the estimator to training data.

#         Parameters
#         ----------
#         X : 3D np.ndarray of shape (n_instances, n_channels, n_timepoints)
#             The training data.
#         y : 1D np.ndarray of shape (n_instances)
#             The target labels for fitting, indices correspond to instance indices in X

#         Returns
#         -------
#         self :
#             Reference to self.
#         """
#         X = torch.from_numpy(X).float()
#         y = torch.from_numpy(y).float()
#         y = y.unsqueeze(1)
#         self.X_mean = X.mean()
#         self.X_std = X.std()
#         self.y_mean = y.mean()
#         self.y_std = y.std()
#         X = (X - self.X_mean) / self.X_std
#         y = (y - self.y_mean) / self.y_std
#         self.hydraboost.fit(X, y)
#         return self


#     def predict(self, X: np.ndarray) -> np.ndarray:
#         """Predicts labels for sequences in X.

#         Parameters
#         ----------
#         X : 3D np.ndarray of shape (n_instances, n_channels, n_timepoints)
#             The training data.

#         Returns
#         -------
#         y : array-like of shape (n_instances)
#             Predicted target labels.
#         """
#         X = torch.from_numpy(X).float()
#         X = (X - self.X_mean) / self.X_std
#         pred = self.hydraboost(X)
#         pred = pred * self.y_std + self.y_mean
#         return pred.squeeze().detach().numpy()
        
        

#     def _more_tags(self) -> dict:
#         return {
#             "X_types": ["3darray"],
#             "equal_length_only": True,
#             "allow_nan": False,
#         }

In [ ]:
# test_regressor(
#     regressor = TSMLWrapperHydraBoost(),
#     regressor_name = "HydraBoost",
# )

# Gridsearch

In [ ]:
from run_regression_experiments import test_regressor, TSMLGridSearchWrapper
from models.random_feature_representation_boosting import HydraBoost


# test_regressor(
#     regressor_name = "HydraBoostGridSearchHoldout",
#     regressor = TSMLGridSearchWrapper(
#                 "holdout",
#                 seed=0,
#                 device="cpu",
#                 modelClass=HydraBoost,
#                 model_param_grid={
#                     "n_layers": [1],          
#                     "init_n_kernels": [8],
#                     "init_n_groups": [64],
#                     "n_kernels": [8],
#                     "n_groups": [64],
#                     "max_num_channels": [3],
#                     "hydra_batch_size": [10000],
#                     "l2_reg": [0.01],               
#                     "l2_ghat": [0.01],         
#                     "boost_lr": [0.5],
#                 },
#                 ),
#     dataset_name = "HouseholdPowerConsumption1",
#     TSER_data_dir = Config.TSER_dir,
#     results_dir = Config.project_dir / "results",
#     resample_id=0,
# )


In [4]:
%run {Config.project_dir / "run_regression_experiments.py"} \
    --dataset_indices 10 \
    --resample_ids 0 \
    --results_dir "{Config.results_dir.as_posix()}" \
    --TSER_dir "{Config.TSER_dir.as_posix()}" \
    --device "cuda" \
    --optuna_or_gridsearch "optuna" \
    --holdout_or_kfold "holdout" \
    --seed 0

/rds/general/user/nz423/home/exploring-hydra-boosting/DATASETS/TSER
/rds/general/user/nz423/home/exploring-hydra-boosting/results_TESTING


[I 2025-02-20 18:07:49,387] A new study created in memory with name: no-name-f2a36449-d674-484f-9f1e-e3860e38ab2f
[I 2025-02-20 18:07:52,698] Trial 0 finished with value: 0.3133780062198639 and parameters: {'n_layers': 6, 'l2_reg': 19.549524484259873, 'l2_ghat': 4.135997393839888, 'boost_lr': 0.5903948646972071}. Best is trial 0 with value: 0.3133780062198639.
[I 2025-02-20 18:07:55,018] Trial 1 finished with value: 0.3158719539642334 and parameters: {'n_layers': 4, 'l2_reg': 7.505241622349544, 'l2_ghat': 0.4222048983149828, 'boost_lr': 0.9025957007038717}. Best is trial 0 with value: 0.3133780062198639.
[I 2025-02-20 18:08:00,306] Trial 2 finished with value: 0.3157699406147003 and parameters: {'n_layers': 10, 'l2_reg': 0.1998246739232945, 'l2_ghat': 56.27949975223143, 'boost_lr': 0.5760054277776141}. Best is trial 0 with value: 0.3133780062198639.
[I 2025-02-20 18:08:03,616] Trial 3 finished with value: 0.3219258785247803 and parameters: {'n_layers': 6, 'l2_reg': 357.7501543082695, '

self.best_params {'n_layers': 6, 'l2_reg': 12.833892418174981, 'l2_ghat': 0.8551846568262816, 'boost_lr': 0.8852516464735346}
66.38944455515707 mse
8.147971806232338 rmse
3699634476053427.5 mape
0.1153636402334477 r2
345120.0 fit time


/rds/general/user/nz423/home/miniforge3/envs/hydraboost/lib/python3.11/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
